In [ ]:
# Le bon 
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import wordnet

# text recognition
import cv2
import pytesseract

# Télécharger les stopwords
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

# Chemin vers le dossier contenant les images
dossier_images = r"/Users/dimitriraymond/Documents/Epitech/Semestre 2/DAT/data/Movie_Poster_Dataset"


# configurations
config = ('-l eng --oem 1 --psm 3')

text = pytesseract.image_to_string(img, config=config)

# Liste pour stocker les résultats
resultats = []

# Parcourir les fichiers et dossiers dans le dossier d'images
for dossier, sous_dossiers, fichiers in os.walk(dossier_images):
    for fichier in fichiers:
        # Vérifier si le fichier est une image JPG
        if fichier.endswith(".jpg") or fichier.endswith(".jpeg"):
            chemin_image = os.path.join(dossier, fichier)

            # Lire l'image
            img = cv2.imread(chemin_image)

            # Reconnaissance de texte
            text = pytesseract.image_to_string(img, config=config)

            # Convertir le texte en minuscules
            text_lower = text.lower()

            # Tokenisation des mots
            tokens = word_tokenize(text_lower)

            # Supprimer les mots vides (stop words)
            stop_words = set(stopwords.words('english'))
            filtered_tokens = [word for word in tokens if word not in stop_words]

            # Identifier les parties du discours des mots
            pos_tags = nltk.pos_tag(filtered_tokens)

            # Extraire les noms (substantifs) et les prénoms (proper nouns)
            nouns = [word for word, pos in pos_tags if pos.startswith('NN')]
            proper_nouns = [word for word, pos in pos_tags if pos.startswith('NNP')]

            # Calculer la fréquence des mots
            word_freq = FreqDist(filtered_tokens)
            most_common_words = word_freq.most_common(10)

            # Identifier les synonymes des mots les plus fréquents
            synonyms = []
            for word, freq in most_common_words:
                synsets = wordnet.synsets(word)
                if synsets:
                    syn = synsets[0].lemmas()[0].name()
                    synonyms.append(syn)

            # Ajouter le titre de l'image et les résultats à la liste des résultats
            titre_image = fichier  # Utiliser le nom du fichier comme titre
            resultats.append([titre_image, nouns, proper_nouns, synonyms])

# Écrire les résultats dans un fichier CSV
chemin_fichier_csv = r"/Users/dimitriraymond/Documents/Epitech/Semestre 2/DAT/resultc/fichier5.csv"
with open(chemin_fichier_csv, mode='w', newline='') as fichier_csv:
    writer = csv.writer(fichier_csv)
    writer.writerow(["Titre de l'image", "Sujets", "Nom", "Synonymes"])  # Écrire l'en-tête du fichier CSV
    writer.writerows(resultats)  # Écrire les résultats

print("Terminé. Les résultats ont été enregistrés dans", chemin_fichier_csv)



# Imprimer les résultats
#print("Sujets du texte (noms) :", nouns)
#print("Prénoms du texte :", proper_nouns)
#print("Mots les plus fréquents :", most_common_words)
#print("Synonymes des mots les plus fréquents :", synonyms)


In [ ]:
import pandas as pd

df = pd.read_csv(r"/Users/dimitriraymond/Documents/Epitech/Semestre 2/DAT/resultc/fichier6.csv")

df['Synonymes'] = df['Synonymes'].str.strip("[]").str.replace("'", "").str.split(", ")

df = pd.concat([df, df['Synonymes'].apply(pd.Series)], axis=1)

df = df.drop('Nom', axis=1)

df = df.drop('Synonymes', axis=1)

df

df.to_csv(r"/Users/dimitriraymond/Documents/Epitech/Semestre 2/DAT/resultc/nouveau_fichier.csv", index=False)
